# Yanal Yahya, software engineering, 197 group

# HA 4

### Pre-requisite:

In [3]:
#!pip install base58 ecdsa prettytable bitcoin==1.1.39

Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py) ... done
  Created wheel for bitcoin: filename=bitcoin-1.1.39-py3-none-any.whl size=28441 sha256=fe83334fb422f2e92f0e26d0097de7656b4a04c27fef221f4f884ecc53d6a0a4
  Stored in directory: /home/yanal/.cache/pip/wheels/83/49/59/01dd3687ca261b46deea85b14c12280c451fc717089775a3ca
Successfully built bitcoin


In [1]:
#Defining Libraries:
import hashlib
import base58
import os
import hashlib
import ecdsa
import binascii
import bitcoin
from ecdsa import SigningKey, SECP256k1
from prettytable import PrettyTable

In [2]:
#Defining Function For Hashing: 1. Sha256 and 2. Ripemd160
def hashing(a):
    first_sha256 = hashlib.sha256(a)
    ripemd160 = hashlib.new("ripemd160")
    ripemd160.update(first_sha256.digest())
    return_0 = ripemd160.digest()
    return_1 = bytes.fromhex("00") + ripemd160.digest()
    return_2 = bytes.fromhex("6f") + ripemd160.digest()
    return return_0, return_1, return_2

def public_to_test(a):
    decoded_address = base58.b58decode(scriptaddress)
    address_1 = decoded_address[1:-4]
    hex_address = bytes.fromhex("C4") + address_1
    checksum_test_multsig = checksum(hex_address)
    test_address = to_base58(checksum_test_multsig)
    return test_address

#Function for Checksum
def checksum(b):
    checksum_full = hashlib.sha256(hashlib.sha256(b).digest()).digest()
    new_checksum = checksum_full[:4]
    return b + new_checksum

#Function to convert to base58    
def to_base58(c):
    return base58.b58encode(c).decode('utf-8')

def compressed_key(d):
    a = d.hex()

    if a[-1] == '0' or a[-1] == '2' or a[-1] == '4' or a[-1] == '6' or a[-1] == '8' or a[-1] == 'a' or a[-1] == 'c' or a[-1] == 'e':
        return bytes.fromhex("02") + d
    
    else:
        return bytes.fromhex("03") + d
    
def generate_address(print_output = True):
    
    #generating Random number of 32 bytes
    private_key = os.urandom(32).hex()
    private_key_from_hex = bytes.fromhex(private_key)

    #Keeping same private key, Generating Public Key
    import ecdsa # pip install ecdsa
    sk = ecdsa.SigningKey.from_string(private_key_from_hex, curve = ecdsa.SECP256k1) 


    #actual publickey to verify messages that singed using own pvt key
    verification_key = sk.verifying_key
    #making Public Key:
    public_key = bytes.fromhex("04") + verification_key.to_string()
    compressed_public_key = compressed_key(verification_key.to_string())
    compressed_public_key_hex = compressed_public_key.hex()[:66]
    #Converting public key to address:
    decoded_pubkey, mainnet_pubkey, testnet_pubkey = hashing(public_key)

    #DECODED PUBLIC KEY WILL BE REQUIRED
    #checksum:
    checksum_main_pubkey = checksum(mainnet_pubkey)
    checksum_test_pubkey = checksum(testnet_pubkey)


    main_address = to_base58(checksum_main_pubkey)
    test_address = to_base58(checksum_test_pubkey)


    if print_output:
        x = PrettyTable()
        x.field_names = ["Discription", "Value"]
        x.add_row(["Private Key, hex", private_key])
        x.add_row(["Public Key, hex", public_key.hex()])
        x.add_row(["Compressed Public Key, hex", compressed_public_key_hex])
        x.add_row(["MainNet Address, base58", main_address])
        x.add_row(["Testnet Address, base58", test_address])
        print(x)
    return(private_key, public_key.hex(), compressed_public_key_hex, main_address, test_address)

1. Generate 3 testnet addresses (address_1, address_2, address_3; should
start with either m or n)

In [3]:
print('''
===============================
First address data
================================''')
address_data_1 = generate_address()
print('''
===============================
Second address data
================================''')
address_data_2 = generate_address()
print('''
===============================
Third address data
================================''')
address_data_3 = generate_address()


First address data
+----------------------------+------------------------------------------------------------------------------------------------------------------------------------+
|        Discription         |                                                               Value                                                                |
+----------------------------+------------------------------------------------------------------------------------------------------------------------------------+
|      Private Key, hex      |                                  1171ff0a9299d180f3dcc00fb981f7f251629f54363678d7a731b737516fa09b                                  |
|      Public Key, hex       | 04fac05cd2ccbc2a8fed4205bb164da54a887d67e496bf362030ab5f0ead75130656a1e8e7ba98cc72fff7efcbe53cb6b484bec6c38874be86e6790ca74f61bb84 |
| Compressed Public Key, hex |                                 02fac05cd2ccbc2a8fed4205bb164da54a887d67e496bf362030ab5f0ead751306                               

2. From prerequisite 1, generate 2-3 MultiSignature address (2 e.g.
private_keys for {addess_1, address_2, address_3} out of three are
required to spend Bitcoins). Note that your 2-3 MultiSignature address
will start with 2.

In [8]:
print("Generating MultiSig Address and Test Address M:N")
m = int(input("Enter N (Max), for example, 2: "))
n = int(input("Enter M (Min), for example, 2: "))

pub_keys = []
for i in range(0,m):
    if i == 1:
        temp = input("Enter {} of {} public key, for example, 0409aa99ce4e7f6b23faf283b5eaab2f6233ac28cbb34cc083957441c5c760fc7b5da64c0975b9f6f8b14f52e1eee5be5750b5ce226b6690bde6438e51211f9ff2: ".format(i+1,m))
    else:
        temp = input("Enter {} of {} public key, for example, 049ea82ac663a76fb872d4a49674762c3f9e60e25d55f7636d76b03efef3d69e838ab749455d3951aa015ccac729ecd9d656e596686af6204534821699e779534b: ".format(i+1,m))
    pub_keys.append(temp)
    
rawscript = bitcoin.mk_multisig_script(pub_keys,n)
descript = bitcoin.deserialize_script(rawscript)
scriptaddress = bitcoin.scriptaddr(rawscript)
print("MultiSig Address: ", scriptaddress)

testnet_address = public_to_test(scriptaddress)
print("MultiSig Testnet Address: ", testnet_address)

print("\nRedeem Script:", rawscript)

Generating MultiSig Address and Test Address M:N
Enter N (Max), for example, 2: 3
Enter M (Min), for example, 2: 2
Enter 1 of 3 public key, for example, 049ea82ac663a76fb872d4a49674762c3f9e60e25d55f7636d76b03efef3d69e838ab749455d3951aa015ccac729ecd9d656e596686af6204534821699e779534b: 04fac05cd2ccbc2a8fed4205bb164da54a887d67e496bf362030ab5f0ead75130656a1e8e7ba98cc72fff7efcbe53cb6b484bec6c38874be86e6790ca74f61bb84
Enter 2 of 3 public key, for example, 0409aa99ce4e7f6b23faf283b5eaab2f6233ac28cbb34cc083957441c5c760fc7b5da64c0975b9f6f8b14f52e1eee5be5750b5ce226b6690bde6438e51211f9ff2: 04c15123e00223c2b8cfb0058531d11efb304fafce6f5bfc8f60dbbf4d878466db3038bf7ea363aaebea69b7f32e82a8800fbd38e7fc95a2d7569c441c8e6aefdf
Enter 3 of 3 public key, for example, 049ea82ac663a76fb872d4a49674762c3f9e60e25d55f7636d76b03efef3d69e838ab749455d3951aa015ccac729ecd9d656e596686af6204534821699e779534b: 04d44f5e00c302729406d4ab56e5aa690489fb162d250566ac873c82dd2b5e012b69325af3778aaff5203f505ac9d869b473c51785ce7c5d8

### Tasks:

### 1.
(1 point) Calculate double Hash_256 of your name. Take two inputs from
the user, i.e., first name and last name. Concatenate string and take
SHA256(SHA256(<first name><lastname>)).


In [5]:
firstName = input('Enter your first name, plz: ')
lastName = input('Enter your last name, plz: ')
fullName = firstName + ' ' + lastName

hash1 = hashlib.sha256(fullName.encode('utf-8')).digest()
doubleHashedFullName = hashlib.sha256(hash1).digest()
doubleHashedFullName = str(binascii.hexlify(doubleHashedFullName),"ascii")
doubleHashedFullName

Enter your first name, plz: Yanal
Enter your last name, plz: Yahya


'5cbadfb70d3e24913f71658d51c6c1d04238d920dd6d78fbda971035c35e61ac'

---

### 2.
(1 point) Get some testnet Bitcoins to your testnet address_1 from any
source. (For example: https://testnet-faucet.mempool.co/,
https://coinfaucet.eu/en/btc-testnet/ or search "bitcoin testnet
faucet"). Provide the Transaction_ID.


$$Testnet Address 1, base58 = mheGBXjbDW65ntJv4764whtTtuqMja3NKc$$

$$Transaction_{ID} = 26a4d1d4be59d20b5e609d438bb3beb852e8a776872fd5174e5f3826f2a6f641$$

---

### 3.
(2 points) Create a new transaction from address_1 and provide
Transaction_ID. Make 2 outputs:
Output_1 : Some bitcoins to address_2.
Output_2: Data transfer with first 4 Bytes double Hash_256 of your name.
You send 0 bitcoins in this output.
Note. Be aware of small transaction fees. You may find
https://blockstream.info/testnet/tx/push useful.


In [18]:
#Function
def reverse(a):
    i = 0
    rev_a = []
    while i != len(a):
        temp_value = a[i] + a[i+1]
        rev_a.append(temp_value)
        i = i + 2
    rev_a.reverse()
    return ''.join(rev_a)

def bitcoin_satoshi_rev_hex(btc):
    temp = float(btc) * (10**8)
    temp_1 = hex(int(temp))[2:]
    temp_2 = reverse(str(temp_1).zfill(16))
    return temp_2

def bytes_padding(temp_no):
    if len(temp_no)/2 != 0:
        return '0' + temp_no
    else:
        return temp_no

def tx_outputs():
    receiver = input("Enter Receiver's Address {}, for example, mp7cUPuKFeR3tm7hMF6Y9PZX89iyHMK67E: ".format(1))
    value = input("Enter Value in BTC (0.0) to send {}, for example, 0.009:".format(receiver))
    value_in_hex_NYB = bitcoin_satoshi_rev_hex(value)
    receiver_address = base58.b58decode(receiver).hex()[2:-8]
    size_receiver = hex(int(len(receiver_address)/2))[2:]
    locking_Script = '76' + 'a9' + size_receiver + receiver_address + '88' + 'ac'
    script_len_2 = hex(int(len(locking_Script)/2))[2:]
    return value_in_hex_NYB + script_len_2 + locking_Script

    
    
def op_return():
    value_return = '0'*16
    #data_1 = b"charley loves heidi"
    data = input("Enter Data to send, for exampe, 'Hello': ")
    text_from_bytes = binascii.b2a_hex(data.encode("utf-8")).decode()
    len_data = hex(int(len(text_from_bytes)/2))[2:]
    partial_data =  '6a'+ bytes_padding(len_data) + text_from_bytes
    return value_return + bytes_padding(hex(int(len(partial_data)/2))[2:]) + partial_data
    
def pushdata(len_r,len_s):
    if len_r == '20' and len_s == '20':
        return str(47)    
    elif len_r == '20' or len_s == '21':
        return str(48)
    elif len_r == '21' or len_s == '20':
        return str(48)
    else:
        return str(49)
    
def pubkey_opcode(a):
    return hex(int(int(len(a))/2))[2:]

In [20]:
#User input:
version_number = '01000000'
number_inputs = '01'
previous_transaction = input("Enter Previous user transaction, for example, 0454bef0e9a178b8f31667dc1cb6ba0fb708d4b745d19a8e3ef6de7b52b6108b: ")
previous_transaction = reverse(previous_transaction)
output_script = '0' + input("Enter previous output (here <= 15), for example, 1: ") + '000000'
script = '1976a914' + input("Enter previous PKH, for example, 76a9145e4f04c3ee1941d665eb1312280ca3a0bbbfd3ba: ") + '88ac'
sequence = 'ffffffff'
tx_ip = version_number + number_inputs + previous_transaction + output_script + script + sequence


#User_output:
Outputs = '02'
out_tx = Outputs + tx_outputs() + op_return() + '0'*8 + '01000000'

#Final Transaction:
raw_transaction = tx_ip + out_tx
print('''Raw transaction: ''', raw_transaction)

#DIGITAL SIGNATURE
unsigned = raw_transaction

temp_bin = binascii.unhexlify(unsigned)
temp_hash = hashlib.sha256(temp_bin).digest()
hash2 = hashlib.sha256(temp_hash).hexdigest()
tx_hash = hashlib.sha256(temp_hash).digest()

private_key = input("Enter Private Key, for example, fbed793f3757d1fddf7d675ed64fa329b28943745b166cc94b30e46c09f778fc: ")
signingkey = ecdsa.SigningKey.from_string(bytes.fromhex(private_key), curve=ecdsa.SECP256k1)
SIG = signingkey.sign_digest(tx_hash, sigencode=ecdsa.util.sigencode_der_canonize)

print("\n\nDigital Signature: ", str(binascii.hexlify(SIG),'ascii'))

#Generating_SigScript:
DER_input =  str(binascii.hexlify(SIG),'ascii')
header = DER_input[0:2]
Sig_Length = DER_input[2:4]
r_integer = DER_input[4:6]
r_length = DER_input[6:8]
r = DER_input[8:8 + 2 * int(r_length,16)]
s_integer = DER_input[8 + 2 * int(r_length,16): 10 + 2 * int(r_length,16)]
s_length = DER_input[10 + 2 * int(r_length,16): 12 + 2 * int(r_length,16)]
s = DER_input[12 + 2 * int(r_length,16):]

pushdata_opcode = pushdata(r_length, s_length)
sighash_code = '01'

publickey = input("Enter Publickey (Compressed or Uncompressed), for example, 0409aa99ce4e7f6b23faf283b5eaab2f6233ac28cbb34cc083957441c5c760fc7b5da64c0975b9f6f8b14f52e1eee5be5750b5ce226b6690bde6438e51211f9ff2: ")
pubkey_pushdata_opcode = pubkey_opcode(publickey)

final_SigScript = pushdata_opcode + header + Sig_Length + r_integer + r_length + r + s_integer + s_length + s + sighash_code + pubkey_pushdata_opcode + publickey
SigScript = pubkey_opcode(final_SigScript) + final_SigScript
print("\nSigScript(hex): ",SigScript,"\n\n\n")

#Broadcasting_Transaction:
broadcasting_tx = version_number + number_inputs + previous_transaction + output_script + SigScript + sequence + out_tx [:-8]
print('''Transaction to broadcast: ''', broadcasting_tx)

Enter Previous user transaction, for example, 0454bef0e9a178b8f31667dc1cb6ba0fb708d4b745d19a8e3ef6de7b52b6108b: 26a4d1d4be59d20b5e609d438bb3beb852e8a776872fd5174e5f3826f2a6f641
Enter previous output (here <= 15), for example, 1: 0
Enter previous PKH, for example, 76a9145e4f04c3ee1941d665eb1312280ca3a0bbbfd3ba: 17521855b12000aa442289da3f173219819b581b
Enter Receiver's Address 1, for example, mp7cUPuKFeR3tm7hMF6Y9PZX89iyHMK67E: msXfkmVMfjMzmj2bEBbe4jedZPYWiUWq8b
Enter Value in BTC (0.0) to send msXfkmVMfjMzmj2bEBbe4jedZPYWiUWq8b, for example, 0.009:0.014
Enter Data to send, for exampe, 'Hello': 5cbadfb7
Raw transaction:  010000000141f6a6f226385f4e17d52f8776a7e852b8beb38b439d605e0bd259bed4d1a426000000001976a91417521855b12000aa442289da3f173219819b581b88acffffffff02c05c1500000000001976a91483c41e740086a521226a39a372f2ef4a4217e07588ac00000000000000000a6a0835636261646662370000000001000000
Enter Private Key, for example, fbed793f3757d1fddf7d675ed64fa329b28943745b166cc94b30e46c09f778fc: 1171ff0a

$$Transaction_{ID} = aa80c073c101760630581d85bfe58a682773d315a8b506a83cf16dfe48d2e9d4$$

---

### 4.
(2 points) From address_2, Send some bitcoins to your 2-of-3
MultiSignature Address.


In [22]:
import base58
import hashlib
import ecdsa
import binascii
from ecdsa import SigningKey, SECP256k1 

#Declaring function:
class variableInteger:
    def reverse_string_in_pair(a_str_ip):
        i = 0
        rev_a = []
        while i != len(a_str_ip):
            #print(a[i] + a[i+1])
            temp_value = a_str_ip[i] + a_str_ip[i+1]
            rev_a.append(temp_value)
            i = i + 2
        rev_a.reverse()
        return ''.join(rev_a)
    
    def padding(b):
        if int(len(b) % 2) == 0:
            return b
        else:
            return '0' + b
    
    def variable_integer(a):
        if type(a) == int:
            temp = a
        else:
            temp = int(len(a)/2)
        
        if temp <= int('0xfc',16):
            return variableInteger.padding(hex(temp)[2:])
        elif temp > int('0xfc',16) and temp <= int('0xffff',16):
            return 'fd' +  variableInteger.reverse_string_in_pair(variableInteger.padding(hex(temp)[2:]))
        elif temp > int('0xffff',16) and temp <= int('0xffffffff',16):
            return 'fe' + variableInteger.reverse_string_in_pair(variableInteger.padding(hex(temp)[2:]))
        elif temp > int('0xffffffff',16) and temp <= int('0xffffffffffffffff',16):
            return 'ff' + variableInteger.reverse_string_in_pair(variableInteger.padding(hex(temp)[2:]))
        
def digital_signature(raw_tx):
    unsigned = raw_tx
    
    temp_bin = binascii.unhexlify(unsigned)
    temp_hash = hashlib.sha256(temp_bin).digest()
    tx_hash = hashlib.sha256(temp_hash).digest()
    private_key = input("Enter Private Key: ")
    signingkey = ecdsa.SigningKey.from_string(bytes.fromhex(private_key), curve=ecdsa.SECP256k1)
    SIG = signingkey.sign_digest(tx_hash, sigencode=ecdsa.util.sigencode_der_canonize)   
    temp_sig = str(binascii.hexlify(SIG),'ascii')
    return temp_sig     

def reverse_string_in_pair(a_str_ip):
    i = 0
    rev_a = []
    while i != len(a_str_ip):
        #print(a[i] + a[i+1])
        temp_value = a_str_ip[i] + a_str_ip[i+1]
        rev_a.append(temp_value)
        i = i + 2
    rev_a.reverse()
    return ''.join(rev_a)

def bitcoin_satoshi_rev_hex(btc):
    temp = float(btc) * (10**8)
    temp_1 = hex(int(temp))[2:]
    temp_2 = reverse_string_in_pair(str(temp_1).zfill(16))
    return temp_2

def tx_inputs_tx_id():
    previous_tx_hash = input("Enter Reference TxId: ")
    previous_tx_hash_reversed = reverse_string_in_pair(previous_tx_hash)
    previous_output_index = input("Enter Previous output index: ")
    previous_output_index = reverse_string_in_pair('{:08d}'.format(int(previous_output_index)))
    return previous_tx_hash_reversed + previous_output_index

def redeem_place_holder():
    redeemScript = input("Enter redeem Script: ")
    return variableInteger.variable_integer(redeemScript) + redeemScript

def tx_outputs(c):
    receiver = input("Enter Receiver's Address {}: ".format(c+1))
    if receiver[0] == 'm' or receiver[0] == 'n': 
        value = input("Enter Value in BTC (0.0) to send {}:".format(receiver))
        value_in_hex_NYB = bitcoin_satoshi_rev_hex(value)
        receiver_address = base58.b58decode(receiver).hex()[2:-8]
        locking_Script = '76' + 'a9' + variableInteger.variable_integer(receiver_address) + receiver_address + '88' + 'ac'
        script_len_2 = variableInteger.variable_integer(locking_Script)
        return value_in_hex_NYB + script_len_2 + locking_Script
    
    elif receiver[0] == '2':
        value = input("Enter Value in BTC (0.0) to send {}:".format(receiver))
        value_in_hex_NYB = bitcoin_satoshi_rev_hex(value)
        receiver_address = base58.b58decode(receiver).hex()[2:-8]
        locking_Script = 'a9' + variableInteger.variable_integer(receiver_address) + receiver_address + '87'
        script_len_2 = variableInteger.variable_integer(locking_Script)
        return value_in_hex_NYB + script_len_2 + locking_Script

In [24]:
def create_new_transaction_3(tx_id, publickey, private_key, n_out, n_data_out):

    version_number = '01000000'
    number_inputs = '01'
    previous_transaction = tx_id
    previous_transaction = reverse(previous_transaction)
    
    previous_output = '1'
    output_script = '0' + previous_output + '000000'
    previous_PKH = input("Enter previous PKH: ")
    script = '1976a914' + previous_PKH + '88ac'
    sequence = 'ffffffff'
    tx_ip = version_number + number_inputs + previous_transaction + output_script + script + sequence


    #User_output:
    Outputs = str(n_out)
    #utxo outputs
    utxo_output = str(1) # the same all time -> hardcoded
    tx_output = ''
    for i in range(0, int(utxo_output)):
            tx_output = tx_outputs(i) + tx_output
    #data output
    data_output = str(n_data_out)
    op_ret = ''
    for i in range(0, int(data_output)):
            op_ret = op_return(i) + op_ret
    out_tx = '0' + Outputs + tx_output + op_ret + '0'*8 + '01000000'

    #Final Transaction:
    raw_transaction = tx_ip + out_tx
    print('''Raw transaction: ''', raw_transaction)

    #DIGITAL SIGNATURE
    unsigned = raw_transaction

    temp_bin = binascii.unhexlify(unsigned)
    temp_hash = hashlib.sha256(temp_bin).digest()
    hash2 = hashlib.sha256(temp_hash).hexdigest()
    tx_hash = hashlib.sha256(temp_hash).digest()

    signingkey = ecdsa.SigningKey.from_string(bytes.fromhex(private_key), curve=ecdsa.SECP256k1)
    SIG = signingkey.sign_digest(tx_hash, sigencode=ecdsa.util.sigencode_der_canonize)

    print("\n\nDigital Signature: ", str(binascii.hexlify(SIG),'ascii'))

    #Generating_SigScript:
    DER_input =  str(binascii.hexlify(SIG),'ascii')
    header = DER_input[0:2]
    Sig_Length = DER_input[2:4]
    r_integer = DER_input[4:6]
    r_length = DER_input[6:8]
    r = DER_input[8:8 + 2 * int(r_length,16)]
    s_integer = DER_input[8 + 2 * int(r_length,16): 10 + 2 * int(r_length,16)]
    s_length = DER_input[10 + 2 * int(r_length,16): 12 + 2 * int(r_length,16)]
    s = DER_input[12 + 2 * int(r_length,16):]

    pushdata_opcode = pushdata(r_length, s_length)
    sighash_code = '01'
 
    pubkey_pushdata_opcode = pubkey_opcode(publickey)

    final_SigScript = pushdata_opcode + header + Sig_Length + r_integer + r_length + r + s_integer + s_length + s + sighash_code + pubkey_pushdata_opcode + publickey
    SigScript = pubkey_opcode(final_SigScript) + final_SigScript
    print("\nSigScript(hex): ",SigScript,"\n\n\n")

    #Broadcasting_Transaction:
    broadcasting_tx = version_number + number_inputs + previous_transaction + output_script + SigScript + sequence + out_tx [:-8]
    print('''Transaction to broadcast: ''', broadcasting_tx)

In [25]:
tx_id_3 = '406cdce2a13f3c0c6846156fa9a1d0a4dcd3bc02648a011ae43a27adb8bab9b2'
priv_key = '0ec1a1909a91214a0f3bc631e8336751773f304d7b000530c0f8e9a6c459cfa4'
public_key = '04c15123e00223c2b8cfb0058531d11efb304fafce6f5bfc8f60dbbf4d878466db3038bf7ea363aaebea69b7f32e82a8800fbd38e7fc95a2d7569c441c8e6aefdf'
# order: tx_id, public key, private key, n_outputs, n_data_outputs
create_new_transaction_3(tx_id_3, public_key, priv_key, 1, 0) 

Enter previous PKH: 83c41e740086a521226a39a372f2ef4a4217e075
Enter Receiver's Address 1: 2N7xXPREcPnUki7MtDFRanzYCxsHsK2i5N2
Enter Value in BTC (0.0) to send 2N7xXPREcPnUki7MtDFRanzYCxsHsK2i5N2:0.013
Raw transaction:  0100000001b2b9bab8ad273ae41a018a6402bcd3dca4d0a1a96f1546680c3c3fa1e2dc6c40010000001976a91483c41e740086a521226a39a372f2ef4a4217e07588acffffffff0120d613000000000017a914a16156d96a8f8f4fe612ab52ccdb3e758c471c2e870000000001000000


Digital Signature:  3045022100d1dc7f6437f403d4aeb4bff9ccc82fcfa3091a71477f7a79cf8bc85ea38389840220465ccf7f7f52fd3f92877b63e05c48292b1d837e84a103657d5ca30d2dbae299

SigScript(hex):  8b483045022100d1dc7f6437f403d4aeb4bff9ccc82fcfa3091a71477f7a79cf8bc85ea38389840220465ccf7f7f52fd3f92877b63e05c48292b1d837e84a103657d5ca30d2dbae299014104c15123e00223c2b8cfb0058531d11efb304fafce6f5bfc8f60dbbf4d878466db3038bf7ea363aaebea69b7f32e82a8800fbd38e7fc95a2d7569c441c8e6aefdf 



Transaction to broadcast:  0100000001b2b9bab8ad273ae41a018a6402bcd3dca4d0a1a96f1546680c3c

$$Transaction_{ID} = 40be6042760846dd121301da05f3db6ee80fa60beea09939989abff201239d9d$$



---

### 5.
(4 points) From 2-of-3 MultiSignature Address. Send some bitcoins back
to address_1.

In [9]:
sig_loop = input("Number of Signatures Required: ")
#Making Raw_Transaction: 
#input Basic
version = '01000000'

#input_multi_Signature:
input_ms = '01' + tx_inputs_tx_id()
redeem_Script = redeem_place_holder()
tx_input_value = input_ms + redeem_Script + 'f'*8

Number of Signatures Required: 2
Enter Reference TxId: 0ecd3f9e588d7064558e4e583866478464e1d02b7aebe91df14523502d019e3f
Enter Previous output index: 0
Enter redeem Script: 524104fac05cd2ccbc2a8fed4205bb164da54a887d67e496bf362030ab5f0ead75130656a1e8e7ba98cc72fff7efcbe53cb6b484bec6c38874be86e6790ca74f61bb844104c15123e00223c2b8cfb0058531d11efb304fafce6f5bfc8f60dbbf4d878466db3038bf7ea363aaebea69b7f32e82a8800fbd38e7fc95a2d7569c441c8e6aefdf4104d44f5e00c302729406d4ab56e5aa690489fb162d250566ac873c82dd2b5e012b69325af3778aaff5203f505ac9d869b473c51785ce7c5d8346f77b12ca7c774f53ae


In [10]:
#output
utxo_output = input("Enter number of outputs: ")
tx_output = ''
for i in range(0, int(utxo_output)):
        tx_output = tx_outputs(i) + tx_output
next_output_data =  variableInteger.variable_integer(int(utxo_output)) + tx_output

locktime = '00000000'
SigHashCode = '01000000'

Enter number of outputs: 1
Enter Receiver's Address 1: mheGBXjbDW65ntJv4764whtTtuqMja3NKc
Enter Value in BTC (0.0) to send mheGBXjbDW65ntJv4764whtTtuqMja3NKc:0.013


In [11]:
tx_to_sign = version + tx_input_value + next_output_data + locktime + SigHashCode
print("Raw Transaction to Sign:")
print(tx_to_sign)

Raw Transaction to Sign:
01000000013f9e012d502345f11de9eb7a2bd0e16484476638584e8e5564708d589e3fcd0e00000000c9524104fac05cd2ccbc2a8fed4205bb164da54a887d67e496bf362030ab5f0ead75130656a1e8e7ba98cc72fff7efcbe53cb6b484bec6c38874be86e6790ca74f61bb844104c15123e00223c2b8cfb0058531d11efb304fafce6f5bfc8f60dbbf4d878466db3038bf7ea363aaebea69b7f32e82a8800fbd38e7fc95a2d7569c441c8e6aefdf4104d44f5e00c302729406d4ab56e5aa690489fb162d250566ac873c82dd2b5e012b69325af3778aaff5203f505ac9d869b473c51785ce7c5d8346f77b12ca7c774f53aeffffffff0120d61300000000001976a91417521855b12000aa442289da3f173219819b581b88ac0000000001000000


In [12]:
# Digital Signature
signatures = ''

for i in range(0,int(sig_loop)):
    signature_temp = digital_signature(tx_to_sign) + '01'
    signature_temp_2 = variableInteger.variable_integer(signature_temp) + signature_temp
    signatures = signatures + signature_temp_2
    print(i+1,signatures)

Enter Private Key: 1171ff0a9299d180f3dcc00fb981f7f251629f54363678d7a731b737516fa09b
1 483045022100f7dd0befbedf8201abbd05d0085172395cf56b18de2ecc2151d6aebea0a9beab02201123d9c3b98b513cbeaf2fc5055e025a6a1d8844ca2aef013b0c1472fec2c00401
Enter Private Key: 0ec1a1909a91214a0f3bc631e8336751773f304d7b000530c0f8e9a6c459cfa4
2 483045022100f7dd0befbedf8201abbd05d0085172395cf56b18de2ecc2151d6aebea0a9beab02201123d9c3b98b513cbeaf2fc5055e025a6a1d8844ca2aef013b0c1472fec2c0040147304402205d882a606ef7d00b4cfbc02d4ff1b02ec35cae38fe3b85169e1540b8737bdce902205ec84b8355ec8ae2e310e47af7a4c5260b7eed7d32de91ab14c3f1b783ddf3c001


In [13]:
SigScript_places = '00' + signatures + '4c' + redeem_Script 
SigScript =  variableInteger.variable_integer(SigScript_places) + SigScript_places
print("SigScript:",SigScript)

SigScript: fd5d0100483045022100f7dd0befbedf8201abbd05d0085172395cf56b18de2ecc2151d6aebea0a9beab02201123d9c3b98b513cbeaf2fc5055e025a6a1d8844ca2aef013b0c1472fec2c0040147304402205d882a606ef7d00b4cfbc02d4ff1b02ec35cae38fe3b85169e1540b8737bdce902205ec84b8355ec8ae2e310e47af7a4c5260b7eed7d32de91ab14c3f1b783ddf3c0014cc9524104fac05cd2ccbc2a8fed4205bb164da54a887d67e496bf362030ab5f0ead75130656a1e8e7ba98cc72fff7efcbe53cb6b484bec6c38874be86e6790ca74f61bb844104c15123e00223c2b8cfb0058531d11efb304fafce6f5bfc8f60dbbf4d878466db3038bf7ea363aaebea69b7f32e82a8800fbd38e7fc95a2d7569c441c8e6aefdf4104d44f5e00c302729406d4ab56e5aa690489fb162d250566ac873c82dd2b5e012b69325af3778aaff5203f505ac9d869b473c51785ce7c5d8346f77b12ca7c774f53ae


In [14]:
Broadcasting_transaction = version + input_ms + SigScript + 'f'*8 + next_output_data + locktime
print(Broadcasting_transaction)

01000000013f9e012d502345f11de9eb7a2bd0e16484476638584e8e5564708d589e3fcd0e00000000fd5d0100483045022100f7dd0befbedf8201abbd05d0085172395cf56b18de2ecc2151d6aebea0a9beab02201123d9c3b98b513cbeaf2fc5055e025a6a1d8844ca2aef013b0c1472fec2c0040147304402205d882a606ef7d00b4cfbc02d4ff1b02ec35cae38fe3b85169e1540b8737bdce902205ec84b8355ec8ae2e310e47af7a4c5260b7eed7d32de91ab14c3f1b783ddf3c0014cc9524104fac05cd2ccbc2a8fed4205bb164da54a887d67e496bf362030ab5f0ead75130656a1e8e7ba98cc72fff7efcbe53cb6b484bec6c38874be86e6790ca74f61bb844104c15123e00223c2b8cfb0058531d11efb304fafce6f5bfc8f60dbbf4d878466db3038bf7ea363aaebea69b7f32e82a8800fbd38e7fc95a2d7569c441c8e6aefdf4104d44f5e00c302729406d4ab56e5aa690489fb162d250566ac873c82dd2b5e012b69325af3778aaff5203f505ac9d869b473c51785ce7c5d8346f77b12ca7c774f53aeffffffff0120d61300000000001976a91417521855b12000aa442289da3f173219819b581b88ac00000000


$$Transaction_{ID} = 6149ac007eee4b530f3f014cc9bcf40591318e6c0230c07fcf99e4fb753cab01$$
